In [ ]:
import sys
import os
import numpy as np
from numpy import loadtxt
import cv2

# caffe layers
caffe_root = '/users/vijay.kumar/caffe/'
sys.path.insert(0, caffe_root + 'python')
sys.path.insert(0, '/users/vijay.kumar/tools/liblinear-2.1/python')

import caffe
from caffe import layers as L
from liblinearutil import *
from utils import *

# enable gpu
caffe.set_mode_gpu()

In [ ]:
annot_dir = '../data/soccer/annot/'
image_dir = '../data/soccer/frames/'
train_split_file = '../data/soccer/train_split.txt'
test_split_file = '../data/soccer/test_split.txt'

In [ ]:
params = read_params()

In [ ]:
# initilaze caffe transformer
transformer = define_transformer()

# load pose nets and the pose estimator
nets, pose_net = load_models()

In [ ]:
import PIL.Image
from cStringIO import StringIO
import IPython.display

def showarray(a, fmt='png'):
    a = np.uint8(a)
    f = StringIO()
    PIL.Image.fromarray(a).save(f, fmt)
    IPython.display.display(IPython.display.Image(data=f.getvalue()))

In [ ]:
def process_files(clips):
    annot_data = {}
    for clip_id in range(len(clips)):
        clip_no = clips[clip_id]
        clip_label_fn = annot_dir + 'clip' + str(clip_no) + '.txt'
        with open(clip_label_fn) as f:
            clip_raw_data = f.readlines()

        clip_data = []
        for _data in clip_raw_data:
            _data = _data.strip().split(' ')
            
            # skip errorenous annotations
            if len(_data) < 10:
                continue    
                
            # skip boxes outside the frame.
            if _data[6] == '1':
                continue
                
            _data[9] = _data[9].replace("\"",'')
            if _data[9] in label_names:
                _data[9] = label_names[_data[9]] 
                clip_data.append(np.concatenate((np.array(_data[0:10]),np.array([clip_no])),axis=0).tolist())
        annot_data[clip_id] = np.array(clip_data).astype(int)                 

    # rename the track-id in clips    
    max_track_id = 0    
    for clip_id in range(len(clips)): #keep the first clip as it is
        annot_data[clip_id][:,0] = max_track_id + annot_data[clip_id][:,0]
        max_track_id = np.max(annot_data[clip_id][:,0]) + 1

    # concatenate the clips    
    annot_data_final = annot_data[0]
    for clip_id in range(1, len(clips)):
        annot_data_final = np.concatenate((annot_data_final, annot_data[clip_id]), axis=0)     
        
    return annot_data_final

In [ ]:
clip_nos = [1, 2, 3, 4, 5, 7, 8, 11, 14, 16, 17, 20, 23, 28, 29, 31, 34, 35, 36, 
            37, 38, 39, 40, 43, 47, 48, 50, 51, 52, 53, 54, 55, 56, 57, 58, 60, 62]

train_clips = loadtxt(train_split_file).astype(int)
test_clips = loadtxt(test_split_file).astype(int)

label_names = {'Manuel_Neuer':0, 'Benedikt_Hoewedes':1, 'Mats_Hummels':2, 'Schweinsteiger':3, 'Mesut_Oezil':4, 
           'Miroslav_Klose':5, 'Thomas_Mueller':6, 'Phillip_Lahm':7, 'Toni_Kroos':8, 'Jerome_Boateng':9, 
           'Christoph_Kramer':10, 'Per_Mertesacker':11, 'Mario_Gotze':12, 'Andre_Schurrle':13, 'Sergio_Romero':14, 
           'Ezequiel_Garay':15, 'Pabli_Zabaleta':16, 'Lucas_Biglia':17, 'Enzo_Perez':18, 'Gonzalo_Huguain':19, 
           'Lionel_Messi':20, 'Javier_MAscherano':21, 'Martin_Demichelis':22, 'Marcos_Rojo':23, 'Ezequiel_Lavezzi':24, 
           'Fernando_Gago':25, 'Sergio_Aguero':26, 'Rodrigo_Palacio':27, 'Referee':28,
          }

train_annot = process_files(train_clips)
test_annot = process_files(test_clips)

In [ ]:
print 'Number of training samples:', train_annot.shape[0]
print 'Number of testing samples:', test_annot.shape[0]

In [ ]:
def extract_regions(image, box):
    
    [x1, y1, x2, y2] = box[1], box[2], box[3], box[4]    
    img_h, img_w, img_c = image.shape

    body = image[max(0, y1): min(img_h, y2),  max(0, x1): min(img_w, x2), :]
    body_h = y2 - y1
    h_x1, h_y1, h_x2, u_x1, u_y1, u_x2 = x1, y1, x2, x1, y1, x2    
    h_y2 = int(h_y1 + 0.4*body_h)
    u_y2 = int(h_y1 + 0.7*body_h)
        
    head = image[max(0, h_y1): min(img_h, h_y2),  max(0, h_x1): min(img_w, h_x2), :]        
    ub = image[max(0, u_y1): min(img_h, u_y2),  max(0, u_x1): min(img_w, u_x2), :]
    
    return head, ub

In [ ]:
# extract training features
num_models = len(nets)
num_train = train_annot.shape[0]
train_features = np.zeros((num_train, num_models, params['FEATSIZE']))
train_labels = np.zeros((num_train,1)) 

for i in range(num_train):        
    
    box_data = train_annot[i]    
    clip_no, frame_no, box_label = box_data[-1], box_data[5], box_data[9]    
    img_name = image_dir + str(clip_no) + '/' + str(frame_no) + '.jpg'
    entire_image = cv2.imread(img_name)      
    head, upper_body = extract_regions(entire_image, box_data)  
    train_features[i] = get_pose_features(transformer, nets, head, upper_body, num_models, params['FEATSIZE'])  
    train_labels[i] = int(box_label)

In [ ]:
# Classifier training
classifiers = train_linear_classifiers(train_features, np.squeeze(train_labels), num_models, params)

In [ ]:
num_test = test_annot.shape[0]
test_labels = []
test_tracks = []
pred_scores = []
for i in range(num_test):            
    box_data = test_annot[i]    
    clip_no, frame_no, box_label, tid = box_data[-1], box_data[5], box_data[9], box_data[0]
    img_name = image_dir + str(clip_no) + '/' + str(frame_no) + '.jpg'
    entire_image = cv2.imread(img_name)      
    head, upper_body = extract_regions(entire_image, box_data)        
    test_feature = get_pose_features(transformer, nets, head, upper_body, num_models, params['FEATSIZE'])                    
    pose_weights = get_pose_weights(transformer, pose_net, upper_body)                                        
    pred_sample_sc = pose_aware_identity_prediction_(classifiers, test_feature, np.array([box_label]), pose_weights,  params, num_models)                       
    pred_scores.append(pred_sample_sc[0]) 
    test_tracks.append(tid)    
    test_labels.append(int(box_label))

In [ ]:
# obtain predicted labels
model_labels = classifiers[0][0].get_labels()
pred_labels = [model_labels[np.argmax(ps)] for ps in pred_scores]

In [ ]:
overall_acc = 100*np.mean(np.squeeze(np.array(test_labels)) == np.squeeze(np.array(pred_labels)))
print 'overall accuracy/Known subject accuracy:', overall_acc
print

In [ ]:
correct_per_player = {}
num_examples_per_player = {}
for i in range(len(test_labels)):
    
        # initiliaze dict
        if test_labels[i] not in num_examples_per_player:
            num_examples_per_player[test_labels[i]] = 0  
            correct_per_player[test_labels[i]] = 0                
            
        num_examples_per_player[test_labels[i]] = num_examples_per_player[test_labels[i]] + 1    
        
        if test_labels[i]==pred_labels[i]:
            correct_per_player[test_labels[i]] = correct_per_player[test_labels[i]] + 1
        
for tl in correct_per_player:
    print int(tl), correct_per_player[tl], num_examples_per_player[tl],100*correct_per_player[tl]/float(num_examples_per_player[tl])  